In [1]:
import keras
from keras.models import load_model
from keras.datasets import cifar10
from keras.layers import Input, Dense, Conv2D, MaxPooling2D, UpSampling2D, BatchNormalization, Activation, Flatten, Dropout
from keras.models import Model
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.optimizers import Adam
from keras.utils import np_utils
import os
import pickle
import numpy as np
import h5py
from keras import backend as K
from keras.optimizers import SGD
from keras.callbacks import LearningRateScheduler
import pandas as pd

Using TensorFlow backend.


In [6]:
input_img = Input(shape=(2048, 2048, 3))
x = Conv2D(16, (7, 7), padding='same')(input_img)
#x = BatchNormalization()(x)
x = Activation('relu')(x)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(32, (5, 5), padding='same')(x)
#x = BatchNormalization()(x)
x = Activation('relu')(x)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(64, (3, 3), padding='same')(x)
#x = BatchNormalization()(x)
x = Activation('relu')(x)

x = Conv2D(16, (7, 7), padding='same',strides=16)(x)
x = Flatten()(x)
x = Dense(128)(x)
x = Activation("relu")(x)
x = Dense(3)(x)
out = Activation("softmax")(x)
model = Model(input_img, out)
model.compile(optimizer='adam', loss='categorical_crossentropy',metrics=["accuracy"])
model.summary()

weights_path = '/mnt/Doc/weight/Autoencoder/256/best_weight_0426.h5' 
assert os.path.exists(weights_path), 'Model weights not found '
f = h5py.File(weights_path)
layer_names = [n.decode('utf8') for n in f.attrs['layer_names']]
weight_value_tuples = []

for k, name in enumerate(layer_names):
    if k >= 9:
        # 全結合層の重みは読み込まない
        break
    g = f[name]
    weight_names = [n.decode('utf8') for n in g.attrs['weight_names']]
    if len(weight_names):
        weight_values = [g[weight_name] for weight_name in weight_names]
        layer = model.layers[k]
        symbolic_weights = layer.trainable_weights + layer.non_trainable_weights
        if len(weight_values) != len(symbolic_weights):
            raise Exception('Layer #' + str(k) +
                            ' (named "' + layer.name +
                            '" in the current model) was found to '
                            'correspond to layer ' + name +
                            ' in the save file. '
                            'However the new layer ' + layer.name +
                            ' expects ' + str(len(symbolic_weights)) +
                            ' weights, but the saved weights have ' +
                            str(len(weight_values)) +' elements.')
        weight_value_tuples += zip(symbolic_weights, weight_values)
K.batch_set_value(weight_value_tuples)
f.close()
print('Model loaded.')

for layer in model.layers[:9]:
        layer.trainable = False


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         (None, 2048, 2048, 3)     0         
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 2048, 2048, 16)    2368      
_________________________________________________________________
activation_16 (Activation)   (None, 2048, 2048, 16)    0         
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 1024, 1024, 16)    0         
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 1024, 1024, 32)    12832     
_________________________________________________________________
activation_17 (Activation)   (None, 1024, 1024, 32)    0         
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 512, 512, 32)      0         
__________

In [7]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         (None, 2048, 2048, 3)     0         
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 2048, 2048, 16)    2368      
_________________________________________________________________
activation_16 (Activation)   (None, 2048, 2048, 16)    0         
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 1024, 1024, 16)    0         
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 1024, 1024, 32)    12832     
_________________________________________________________________
activation_17 (Activation)   (None, 1024, 1024, 32)    0         
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 512, 512, 32)      0         
__________

/usr/local/lib/python3.5/dist-packages/keras/engine/training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


In [8]:
class ImageDataGenerator(object):
    def __init__(self):
        self.reset()

    def reset(self):
        self.images = []
        self.labels = []
    #file_pathはメタCSVでーたのこと
    def flow_from_directory(self, file_path, classes, batch_size=1):
        # LabelEncode(classをint型に変換)するためのdict
        blunc=np.empty((batch_size, 2048, 2048, 3), np.float32)
        blunc_l=np.empty((batch_size), np.float32)
        
        #LabelEncode(classをint型に変換)するためのdict
        classes = {v: i for i, v in enumerate(sorted(classes))}
        file_path = pd.read_csv(file_path)
        boss_dir = "/mnt/Doc"
        while True:
            # ディレクトリから画像のパスを取り出す
            for i in range(len(file_path)):
                # 画像を読み込みRGBへの変換、Numpyへの変換を行い、配列(self.iamges)に格納
                path = file_path.loc[i]["path1"]
                #print(i)
                with Image.open(path) as f:
                    self.images.append((np.asarray(f.convert('RGB'), dtype=np.float32))/255)  
                # ファイル名からラベルを取り出し、配列(self.labels)に格納
                #_, y = path.stem.split('_')
                label = file_path.loc[i]["type"]
                self.labels.append(to_categorical(classes[label],len(classes)))
                # ここまでを繰り返し行い、batch_sizeの数だけ配列(self.iamges, self.labels)に格納
                # batch_sizeの数だけ格納されたら、戻り値として返し、配列(self.iamges, self.labels)を空にする
                if len(self.images)== batch_size:
                    print(len(self.images))
                    inputs = np.asarray(self.images,dtype=np.float32)
                    targets = np.asarray(self.labels,dtype=np.float32)
                    print(len(self.images))
                    self.reset()
                    yield inputs, targets

In [ ]:
test_dir = pathlib.Path('/path/to/test/')
x = pd.read_csv(\"/mnt/Doc/2048pix_tissue_all/RGBmean<190.csv")
train_dir = "/mnt/Doc/2048pix_tissue_all/train_1000.csv"
test_dir = "/mnt/Doc/2048pix_tissue_all/val_300.csv"
train_len = len(pd.read_csv(train_dir))
print(train_len)
test_len = len(pd.read_csv(test_dir))
test_datagen = ImageDataGenerator(),
train_datagen = ImageDataGenerator()
classes =["PI","PP","TRU"],
batchsize = 3
model.fit_generator(
    generator=train_datagen.flow_from_directory(train_dir, classes),
    steps_per_epoch=999/3,\n",
    epochs=3000,\n",
    verbose=2,\n",
    #validation_data=test_datagen.flow_from_directory(test_dir, classes),\n",
    #validation_steps=int(test_len / batchsize)\n",
    )

In [9]:
pd.read_csv("/home/kanayalab/Documents/2048pix tissue all/train_1000.csv")

AttributeError: module 'pandas' has no attribute 'read_csv'